In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
orders = pd.read_csv('orders.csv')
order_product_prior = pd.read_csv('order_products__prior.csv')
order_product_train = pd.read_csv('order_products__train.csv')
products = pd.read_csv('products.csv')

In [3]:
order_products = order_product_prior.append(order_product_train)

C:\Users\MUH0351\AppData\Local\Temp\ipykernel_2516\3200147951.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  order_products = order_product_prior.append(order_product_train)


In [4]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [5]:
order_products.product_id.nunique()

49685

In [6]:
product_counts = order_products.groupby('product_id')['order_id'].count().reset_index().rename(columns = {'order_id':'orders'})
product_counts = product_counts.sort_values('orders', ascending=False)[0:200].reset_index(drop = True)
product_counts = product_counts.merge(products, on = 'product_id', how = 'left')
product_counts.head(10)

,product_id,orders,product_name,aisle_id,department_id
0,24852,491291,Banana,24,4
1,13176,394930,Bag of Organic Bananas,24,4
2,21137,275577,Organic Strawberries,24,4
3,21903,251705,Organic Baby Spinach,123,4
4,47209,220877,Organic Hass Avocado,24,4
5,47766,184224,Organic Avocado,24,4
6,47626,160792,Large Lemon,24,4
7,16797,149445,Strawberries,24,4
8,26209,146660,Limes,24,4
9,27845,142813,Organic Whole Milk,84,16


In [7]:
freq_products = list(product_counts.product_id)
len(freq_products)

200

In [8]:
order_products = order_products[order_products['product_id'].isin(freq_products)]
order_products.shape

(10344052, 4)

In [9]:
order_products = order_products.merge(products, on = 'product_id', how='left')
order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,2,28985,2,1,Michigan Organic Kale,83,4
2,2,17794,6,1,Carrots,83,4
3,3,33754,1,1,Total 2% with Strawberry Lowfat Greek Strained...,120,16
4,3,24838,2,1,Unsweetened Almondmilk,91,16


In [10]:
transactions=pd.get_dummies(order_products.drop(['product_id','add_to_cart_order','reordered','aisle_id','department_id'], axis=1))
transactions.set_index(['order_id'], inplace=True)
transactions= transactions.groupby(level=0).sum()
transactions

,product_name_100% Raw Coconut Water,product_name_100% Recycled Paper Towels,product_name_100% Whole Wheat Bread,product_name_2% Reduced Fat Milk,product_name_Apple Honeycrisp Organic,product_name_Asparagus,product_name_Baby Spinach,product_name_Bag of Organic Bananas,product_name_Banana,product_name_Bartlett Pears,...,product_name_Unsweetened Almondmilk,product_name_Unsweetened Original Almond Breeze Almond Milk,product_name_Unsweetened Vanilla Almond Milk,product_name_Vanilla Almond Breeze Almond Milk,product_name_Watermelon Chunks,product_name_Whipped Cream Cheese,product_name_Whole Milk,product_name_Yellow Bell Pepper,product_name_Yellow Onions,"product_name_YoKids Squeezers Organic Low-Fat Yogurt, Strawberry"
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421078,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3421080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3421081,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2655924 entries, 1 to 3421083
Columns: 200 entries, product_name_100% Raw Coconut Water to product_name_YoKids Squeezers Organic Low-Fat Yogurt, Strawberry
dtypes: uint8(200)
memory usage: 526.8 MB


In [15]:
transactions.size

531184800

In [16]:
transactions.shape

(2655924, 200)

In [17]:
freq_itemsets = apriori(transactions, min_support=0.02, use_colnames=True, low_memory=True)
freq_itemsets.head()

C:\Users\MUH0351\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.023763,(product_name_100% Whole Wheat Bread)
1,0.032859,(product_name_Apple Honeycrisp Organic)
2,0.026790,(product_name_Asparagus)
3,0.148698,(product_name_Bag of Organic Bananas)
4,0.184979,(product_name_Banana)


In [18]:
freq_itemsets.tail()

,support,itemsets
56,0.024384,"(product_name_Bag of Organic Bananas, product_..."
57,0.024361,"(product_name_Bag of Organic Bananas, product_..."
58,0.020938,"(product_name_Banana, product_name_Organic Avo..."
59,0.020104,"(product_name_Banana, product_name_Organic Bab..."
60,0.021962,"(product_name_Banana, product_name_Organic Str..."


In [19]:
freq_itemsets.shape

(61, 2)

In [23]:
rules = association_rules(freq_itemsets, metric="lift", min_threshold=1)
rules.sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(product_name_Bag of Organic Bananas),(product_name_Organic Hass Avocado),0.148698,0.083164,0.024384,0.163981,1.971781,0.012017,1.096669,0.578930
1,(product_name_Organic Hass Avocado),(product_name_Bag of Organic Bananas),0.083164,0.148698,0.024384,0.293199,1.971781,0.012017,1.204445,0.537549
5,(product_name_Organic Avocado),(product_name_Banana),0.069363,0.184979,0.020938,0.301866,1.631892,0.008108,1.167428,0.416075
4,(product_name_Banana),(product_name_Organic Avocado),0.184979,0.069363,0.020938,0.113194,1.631892,0.008108,1.049425,0.475097
2,(product_name_Bag of Organic Bananas),(product_name_Organic Strawberries),0.148698,0.103759,0.024361,0.163832,1.578957,0.008933,1.071842,0.430717
3,(product_name_Organic Strawberries),(product_name_Bag of Organic Bananas),0.103759,0.148698,0.024361,0.234787,1.578957,0.008933,1.112504,0.409121
6,(product_name_Banana),(product_name_Organic Baby Spinach),0.184979,0.094771,0.020104,0.108683,1.146794,0.002573,1.015608,0.157056
7,(product_name_Organic Baby Spinach),(product_name_Banana),0.094771,0.184979,0.020104,0.212133,1.146794,0.002573,1.034465,0.141405
8,(product_name_Banana),(product_name_Organic Strawberries),0.184979,0.103759,0.021962,0.118728,1.144263,0.002769,1.016985,0.154689
9,(product_name_Organic Strawberries),(product_name_Banana),0.103759,0.184979,0.021962,0.211665,1.144263,0.002769,1.033851,0.140671
